In [1]:
import numpy as np
import pandas as pd

In [2]:
from scipy.spatial.distance import squareform, pdist

In [3]:
from scipy.sparse.csgraph import minimum_spanning_tree

In [4]:
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt

In [5]:
import sys

In [6]:
from sklearn.metrics.pairwise import pairwise_distances

In [7]:
from sklearn.metrics import f1_score

In [8]:
from sklearn.metrics.cluster import fowlkes_mallows_score

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import cut_tree

In [10]:
from sklearn.cluster import AgglomerativeClustering

In [11]:
def labelClusters(clusters, target):
    mySet = set(clusters)
    resultList = []
    for i in mySet:
        myClasses = []
        for j in range(len(clusters)):
            if (clusters[j] == i):
                myClasses.append(target[j])
        resultList.append(max(set(myClasses), key=myClasses.count))
    
    mySetArray = []
    for i in mySet:
        mySetArray.append(i)
        
    labels = []  
    for i in range(len(clusters)):
        for j in range(len(mySetArray)):
            if (clusters[i] == mySetArray[j]):
                labels.append(resultList[j])
    return labels

In [12]:
def gini2(arr):
    
    somai = 0
    somatot = arr[-1]
    for i in range(len(arr)-1):
        somaj = 0
        for j in range(i+1, len(arr)):
            somaj += abs(arr[i] - arr[j])
        somai += somaj
        somatot += arr[i]
    mult = (len(arr) - 1) * somatot
    
    return (somai/mult)

In [13]:
def counting(arr):
    mySet = set(arr)
    myList = []
    myNewSet = []
    for i in mySet:
        myNewSet.append(i)
        
    for i in range(len(myNewSet)):
        myList.append(0)
        for j in arr:
            if (myNewSet[i]==j):
                myList[i] += 1
    return myList

In [14]:
def menorCluster(clusters):
    mySet = set(clusters)
    minimo = len(clusters)
    
    for i in mySet:
        if((clusters.count(i)) < minimo):
            minimo = clusters.count(i)
            
    myList = []
    
    for j in range(len(clusters)):
        if(clusters.count(clusters[j])==minimo):
            myList.append(j)
            
    return myList

In [15]:
def genieLinkage2(data, limiar, clusterNumber):
    
    clusters = {}
    row_index = 0
    col_index = 0
    array = []
    

    for n in range(data.shape[0]):
        array.append(n)
    
        
    clusters[0] = array.copy()
    
    dists = []
    
    for i in range(0, data.shape[0]):
        for j in range(i, data.shape[1]):
            dists.append(([data[i][j], i, j]))
    
    dists.sort(key=lambda x: x[0])
    pos = 0

    for k in range(1, data.shape[0]):
        min_val = sys.maxsize
        
        if(gini2(counting(clusters[k-1])) <= limiar):
            for i in range(pos, len(dists)):
                if(dists[i][0] == data[dists[i][1]][dists[i][2]]):
                    pos = i
                    min_val = dists[i][0]
                    row_index = dists[i][1]
                    col_index = dists[i][2]
                    break
        else:
            iList = menorCluster(clusters[k-1])
            for i in iList:
                for j in range(0, data.shape[0]):
                    if(data[i][j]<min_val):
                        min_val = data[i][j]
                        row_index = i
                        col_index = j

        array = clusters[k-1].copy()
        minimum = min(array[row_index],array[col_index])
        maximum = max(array[row_index],array[col_index])
        
        for n in range(len(array)):
            if(array[n]==maximum):
                array[n] = minimum
        clusters[k] = array.copy()
        
        for i in range(len(array)):
            for j in range(len(array)):
                if(array[i]==array[j]):
                    data[i][j] = sys.maxsize
                    
        if (len(set(clusters[k])) == clusterNumber):
            return clusters[k]

In [16]:
def genie2(dataset, target, limiar, clusterNumber):

    dataX = dataset.values
    dataY = target[0]
    
    dataDistances = pairwise_distances(dataX,metric='euclidean')
    np.fill_diagonal(dataDistances,sys.maxsize)
    
    clusters = genieLinkage2(dataDistances, limiar, clusterNumber)
    
    labels = labelClusters(clusters, dataY)
    
    fmScore = fowlkes_mallows_score(dataY, labels)
    
    return fmScore

In [17]:
a1 = pd.read_table("datasets/a1.data", header=None, delim_whitespace=True)
iris5 = pd.read_table("datasets/iris5.data", header=None, delim_whitespace=True)
flame = pd.read_table("datasets/flame.data", header=None, delim_whitespace=True)
pathbased = pd.read_table("datasets/pathbased.data", header=None, delim_whitespace=True)
unbalance = pd.read_table("datasets/unbalance.data", header=None, delim_whitespace=True)

In [18]:
a1labels = pd.read_table("labels/a1.labels", header=None, delim_whitespace=True)
iris5labels = pd.read_table("labels/iris5.labels", header=None, delim_whitespace=True)
flamelabels = pd.read_table("labels/flame.labels", header=None, delim_whitespace=True)
pathbasedlabels = pd.read_table("labels/pathbased.labels", header=None, delim_whitespace=True)
unbalancelabels = pd.read_table("labels/unbalance.labels", header=None, delim_whitespace=True)

In [19]:
#só executar para limpar os results

result = pd.DataFrame(columns = ["single","complete", "ward", "average","gini_02","gini_03","gini_04","gini_05","gini_06"])
result.to_csv("results/result.csv")

In [20]:
def linkageScore(method, data, labels, k):
    dataDistances = pairwise_distances(data, metric='euclidean')
    agg = AgglomerativeClustering(n_clusters=k, linkage=method)
    kClusters = agg.fit_predict(dataDistances)
    resultLabels = labelClusters(kClusters, labels)
    fmScore = fowlkes_mallows_score(labels, resultLabels)
    return fmScore

def computeScore(base):
    result = pd.read_csv("results/result.csv", index_col=0)
    
    done = 0
    for name in result.index:
        if (name==base):
            done = 1
    
    if(done==0):
        data = pd.read_table("datasets/"+base+".data", header=None, delim_whitespace=True)
        labels = pd.read_table("labels/"+base+".labels", header=None, delim_whitespace=True)
        k = len(set(labels[0]))

        resultData = []
        for i in range(1,7):
            if (i==1):
                resultData.append((genie2(data, labels, 1, k)))
                methods = ["complete", "ward", "average"]
                for method in methods:
                    resultData.append(linkageScore(method, data, labels[0], k))
            else:
                resultData.append((genie2(data, labels, i/10, k)))


        size = len(result)
        result.loc[size] = resultData
        result = result.rename({size:base})
        result.to_csv("results/result.csv")

In [21]:
computeScore("flame")
# computeScore("jain")
# computeScore("flame")
# computeScore("R15")
# computeScore("spiral")
# computeScore("pathbased")
# computeScore("Compound")
# computeScore("Aggregation")
# computeScore("g2-2-100")

In [ ]:
result = pd.read_csv("results/result.csv", index_col=0)
result

In [ ]:
a = pd.concat([iris5, iris5labels], axis=1)
print('iris5')
result = {}
for i in range(10):
    b = a.sample(frac=1)
    c = b.iloc[:, :-1]
    d = b.iloc[:, -1:]
    k = len(set(d[0]))
    print('complete ', linkageScore("complete", c, d[0], k),'ward ', linkageScore('ward', c, d[0], k),'average ', linkageScore('average', c, d[0], k))